In [15]:
import nltk
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import json
import re
import string
#import emoji

In [16]:
path = 'dataset_tweets_WHO.txt'

#convert the text to json
with open(path) as f:
    tweets_json = json.load(f)
    

In [17]:
#inspecting the tweet. all keys are strings btw
tweets_json['0']

{'created_at': 'Wed Oct 13 09:15:58 +0000 2021',
 'id': 1448215930178310144,
 'id_str': '1448215930178310144',
 'full_text': "It's International Day for Disaster Risk Reduction\n\n#OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response.\n\nStart learning today &amp; be #Ready4Response:\n👉 https://t.co/hBFFOF0xKL https://t.co/fgZY22RWuS",
 'truncated': False,
 'display_text_range': [0, 274],
 'entities': {'hashtags': [{'text': 'OpenWHO', 'indices': [52, 60]},
   {'text': 'Ready4Response', 'indices': [232, 247]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/hBFFOF0xKL',
    'expanded_url': 'https://bit.ly/3wCa0Dr',
    'display_url': 'bit.ly/3wCa0Dr',
    'indices': [251, 274]}],
  'media': [{'id': 1448215398814560259,
    'id_str': '1448215398814560259',
    'indices': [275, 298],
    'media_url': 'http://pbs.twimg.com/ext_tw_video_thumb/1448215398814560259/pu/img/0CO

In [18]:
#remove emojis from a string text
def remove_emoji(line):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', str(line))

def remove_punct(line):
    return line.translate(str.maketrans('', '', string.punctuation))

def build_terms(line):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
    line =  line.lower() ## Transform in lowercase
    line = remove_emoji(line) ## remove all emojis
    line = re.sub(r"\S*https?:\S*", "", line) ## remove all urls
    line = remove_punct(line) #remove punctuation
    
    line = line.split() ## Tokenize the text to get a list of terms
    print(line)
    line = [word for word in line if word not in stop_words] ##eliminate the stopwords (HINT: use List Comprehension)
    line = [stemmer.stem(word) for word in line] ## perform stemming (HINT: use List Comprehension)

    line = [word.replace('#', '') if word[0] == '#' else word for word in line ]
   
    
    """to discuss, should we remove hashtags? 
    yes because if we don't, then something e.x. #covid and covid will have different term frequencies. A document that only
    contains '#covid' is probably very relevant for covid, but will get a tf of 0 if we are searching using the query 'covid'
    """
    
    ## END CODE
    return line

In [19]:
for key in tweets_json:
    key = str(key)
    tweets_json[key]['full_text'] = build_terms(tweets_json[key]['full_text'])

['its', 'international', 'day', 'for', 'disaster', 'risk', 'reduction', 'openwho', 'has', 'launched', 'a', 'multitiered', 'core', 'curriculum', 'to', 'help', 'equip', 'you', 'with', 'the', 'competencies', 'needed', 'to', 'work', 'within', 'public', 'health', 'emergency', 'response', 'start', 'learning', 'today', 'amp', 'be', 'ready4response']
['covid19', 'has', 'shown', 'how', 'health', 'emergencies', 'and', 'disasters', 'affect', 'entire', 'communities', '–', 'especially', 'those', 'with', 'weak', 'health', 'systems', 'and', 'vulnerable', 'populations', 'like', 'migrants', 'indigenous', 'peoples', 'and', 'those', 'living', 'in', 'fragile', 'humanitarian', 'conditions']
['its', 'international', 'day', 'for', 'disaster', 'risk', 'reduction', 'to', 'better', 'respond', 'to', 'emergencies', 'countries', 'must', 'invest', 'in', 'health', 'care', 'systems', 'achieve', 'gender', 'equity', 'protect', 'marginalised', 'groups', 'ensure', 'ready', 'amp', 'equitable', 'access', 'to', 'supplies', 

['drtedros', 'who', 'has', 'said', 'consistently', 'that', 'we', 'need', 'new', 'tools', 'to', 'get', 'malaria', 'control', 'back', 'on', 'track', 'two', 'years', 'ago', 'who', 'and', 'our', 'partners', 'began', 'a', 'pilot', 'programme', 'to', 'roll', 'out', 'this', 'vaccine', 'in', 'and', 'drtedros', 'endmalaria']
['drtedros', 'but', 'globally', 'progress', 'has', 'stalled', 'at', 'an', 'unacceptably', 'high', 'level', 'with', 'more', 'than', '200', 'million', 'cases', 'and', '400', 'thousand', 'deaths', 'every', 'year', 'twothirds', 'of', 'those', 'deaths', 'are', 'children', 'under', 'five', 'in', 'africadrtedros', 'endmalaria']
['drtedros', 'we', 'have', 'made', 'incredible', 'progress', 'in', 'the', 'fight', 'against', 'malaria', 'in', 'the', 'past', '2', 'decades', 'since', '2000', 'malaria', 'deaths', 'have', 'fallen', 'by', 'more', 'than', 'half', 'and', 'we', 'have', 'succeeded', 'in', 'eliminating', 'malaria', 'from', 'many', 'parts', 'of', 'the', 'worlddrtedros', 'endmalari

['drtedros', 'emmanuelmacron', 'actaccelerator', 'i', 'would', 'also', 'like', 'to', 'thank', 'the', 'auvergnerhonealpes', 'region', 'the', 'lyon', 'metropole', 'and', 'the', 'city', 'of', 'lyon', 'for', 'your', 'strong', 'support', 'investment', 'and', 'hospitality', 'for', 'bringing', 'us', 'to', 'this', 'momentdrtedros', 'whoacademy']
['drtedros', 'emmanuelmacron', 'actaccelerator', 'under', 'president', 'emmanuelmacrons', 'leadership', 'the', 'government', 'of', 'france', 'has', 'been', 'a', 'driver', 'and', 'principal', 'investor', 'in', 'the', 'whoacademy', 'and', 'has', 'done', 'much', 'to', 'guide', 'its', 'developmentdrtedros']
['drtedros', 'emmanuelmacron', 'actaccelerator', 'this', 'goes', 'back', 'to', '1851', 'when', 'the', 'first', 'international', 'sanitary', 'conference', 'was', 'held', 'in', 'paris', 'later', 'the', 'first', 'international', 'office', 'of', 'public', 'hygiene', '–', 'the', 'forerunner', 'of', 'the', 'who', '–', 'was', 'founded', 'in', 'paris', 'in', '1

['rt', 'drtedros', 'i', 'also', 'congratulate', 'oman', 'for', 'organizing', 'covid19', 'vaccination', 'process', 'so', 'efficiently', 'and', 'for', 'making', 'sure', 'that', 'it', 'is', 'a', 'smoot…']
['rt', 'drtedros', 'glad', 'to', 'be', 'in', 'i', 'thank', 'covid19', 'vaccinators', 'for', 'walking', 'me', 'through', 'the', 'vaccination', 'process', 'omanconvention', 'from', 'storage…']
['rt', 'whoukraine', 'tonight', 'whoukraine', 'in', 'partnership', 'with', 'mohukraine', 'lit', 'up', 'the', 'ministry', 'of', 'health', 'building', 'in', 'orange', 'to', 'mark', 'worldpati…']
['rt', 'who', 'whoilo', 'study', 'reveals', 'that', 'almost', '2⃣', 'million', 'people', 'were', 'killed', 'by', 'workrelated', 'diseases', 'amp', 'injuries', 'in', '2016', 'find', 'out', 'more', 'in…']
['rt', 'drtedros', 'this', 'years', 'worldpatientsafetyday', 'focuses', 'on', 'safe', 'maternal', 'amp', 'newborn', 'care', 'childbirth', 'should', 'be', 'a', 'time', 'of', 'celebration', 'but', 'f…']
['rt', 'dr

['vaccines', 'give', 'us', 'a', 'high', 'level', 'of', 'protection', 'from', 'serious', 'illness', 'amp', 'death', 'from', 'covid19', 'variants', 'including', 'delta', 'the', 'more', 'people', 'are', 'vaccinated', 'the', 'less', 'opportunity', 'the', 'virus', 'has', 'to', 'spread', 'amp', 'develop', 'new', 'variants']
['we', 'all', 'have', 'sexual', 'health', 'needs', 'which', 'change', 'across', 'our', 'lifespan', '–', 'not', 'only', 'during', 'our', 'reproductive', 'years', 'learn', 'what', 'who', 'and', 'hrpresearch', 'are', 'doing', 'to', 'promote', 'sexual', 'health', 'and', 'wellbeing', 'for', 'all']
['did', 'you', 'know', 'you', 'can', 'have', 'a', 'sexually', 'transmitted', 'infection', 'without', 'showing', 'any', 'symptoms', 'it’s', 'worth', 'getting', 'tested', 'sti', 'kits', 'allow', 'people', 'to', 'collect', 'their', 'own', 'samples', 'for', 'testing', 'even', 'if', 'they', 'are', 'not', 'able', 'to', 'get', 'to', 'a', 'clinic']
['every', 'day', 'people', 'with', 'disabil

...

['healthy', 'amp', 'sustainable', 'diets', 'are', 'a', 'no', '1', 'priority', 'for', 'youth', 'on', 'international', 'youthday', 'let’s', 'practice', 'these', '5', 'keys', 'to', 'safer', 'food', 'tip', '5⃣', 'use', 'safe', 'water', 'amp', 'raw', 'materials', 'youthlead']
['healthy', 'amp', 'sustainable', 'diets', 'are', 'a', 'no', '1', 'priority', 'for', 'youth', 'on', 'international', 'youthday', 'let’s', 'practice', 'these', '5', 'keys', 'to', 'safer', 'food', 'tip', '4⃣', 'always', 'keep', 'food', 'at', 'safe', 'temperatures', 'youthlead']
['healthy', 'amp', 'sustainable', 'diets', 'are', 'a', 'no', '1', 'priority', 'for', 'youth', 'on', 'international', 'youthday', 'let’s', 'practice', 'these', '5', 'keys', 'to', 'safer', 'food', 'tip', '3⃣', 'always', 'cook', 'your', 'food', 'thoroughly', 'youthlead']
['healthy', 'amp', 'sustainable', 'diets', 'are', 'a', 'no', '1', 'priority', 'for', 'youth', 'on', 'international', 'youthday', 'let’s', 'practice', 'these', '5', 'keys', 'to', 'saf

['rt', 'who', '‘there’s', 'no', 'magic', 'dust’', '—', 'act', 'now', 'to', 'protect', 'yourself', 'and', 'loved', 'ones', 'from', 'covid19', 'drmikeryan', 'explains']
['‘there’s', 'no', 'magic', 'dust’', '—', 'act', 'now', 'to', 'protect', 'yourself', 'and', 'loved', 'ones', 'from', 'covid19', 'drmikeryan', 'explains']
['rt', 'drtedros', 'almost', '4', 'million', 'covid19', 'cases', 'were', 'reported', 'to', 'who', 'last', 'week', 'many', 'of', 'these', 'were', 'driven', 'by', 'the', 'highlytransmissible', 'delta…']
['rt', 'drtedros', 'last', 'month', 'we', 'announced', 'the', 'setting', 'up', 'of', 'a', 'technology', 'transfer', 'hub', 'for', 'mrna', 'vaccines', 'in', 'as', 'part', 'of', 'our', 'efforts', 'to', 'scale…']
['there', 'are', '3⃣', 'things', 'to', 'do', 'globally', 'to', 'reduce', 'social', 'isolation', 'and', 'loneliness', 'create', 'a', 'coalition', 'to', 'increase', 'political', 'commitment', 'improve', 'research', 'and', 'evidence', 'for', 'what', 'works', 'scale', 'up

['about', '15', 'of', 'the', '’s', 'population', 'live', 'with', 'a', 'disability', 'this', 'partnership', 'will', 'provide', 'a', 'platform', 'for', 'more', 'persons', 'with', 'disabilities', 'to', 'participate', 'in', 'sport', 'amp', 'ensure', 'they', 'have', 'the', 'care', 'amp', 'technologies', 'they', 'need', 'to', 'fulfil', 'their', 'potential', 'read', 'more']
['the', 'neurological', 'disorders', 'causing', 'the', 'biggest', 'health', 'burden', 'are', 'dementia', 'epilepsy', 'meningitis', 'migraine', 'stroke', 'brain', 'health', 'conditions', 'emerge', 'throughout', 'life', 'amp', 'are', 'characterized', 'by', 'disruptions', 'in', 'normal', '🧠growth', 'andor', 'brain', 'functioning']
['its', 'worldbrainday', '🧠', 'multiple', 'sclerosis', 'affects', '28', 'million', 'people', 'every', '5⃣', 'minutes', 'someone', 'is', 'diagnosed', 'treatment', 'can', 'slow', 'disease', 'progression', 'and', 'improve', 'quality', 'of', 'life', 'yet', 'access', 'to', 'treatment', 'is', 'limited', '

['rt', 'drtedros', 'as', 'the', 'g20org', 'communique', 'says', 'vaccination', 'against', 'covid19', 'is', 'a', 'public', 'good', 'doses', 'must', 'be', 'equitably', 'shared', 'via', 'actacceler…']
['rt', 'drtedros', 'glad', 'to', 'see', 'g20org', 'finance', 'ministers', 'note', 'the', 'threats', 'from', 'covid19', 'variants', 'amp', 'vital', 'need', 'for', 'private', 'amp', 'public', 'sector', 'player…']
['rt', 'drtedros', 'the', 'time', 'is', 'indeed', 'now', 'to', 'do', 'it', 'right', 'because', 'covid19', 'is', 'not', 'over', 'until', 'it’s', 'over', 'wearamask', 'washyourhands', 'keep', 'a', 'safe', 'dis…']
['for', 'your', 'sexual', 'health', 'use', 'a', 'condom', 'consistently', 'and', 'correctly', 'to', 'prevent', 'unintended', '🤰', 'hiv', 'and', 'other', 'sexually', 'transmitted', 'infections', 'selfcare', 'month']
['rt', 'drtedros', 'we', 'welcome', 'the', 'un', 'security', 'council’s', 'decision', 'to', 'extend', 'the', 'crossborder', 'mechanism', 'in', 'northwest', 'syria', 

['sids', 'face', 'challenges', 'to', 'provide', 'effective', 'affordable', 'amp', 'climateresilient', 'health', 'systems', 'to', 'serve', 'populations', 'which', 'are', 'often', 'geographically', 'dispersed', 'living', 'in', 'precarious', 'conditions', 'amp', 'are', 'exposed', 'to', 'increasingly', 'extreme', 'weather', 'events', 'climatechange']
['sealevel', 'rise', 'threatens', 'fragile', 'healthcare', 'facilities', 'in', 'sids', 'which', 'are', 'often', 'located', 'in', 'coastal', 'or', 'lowlying', 'areas', 'the', 'lives', 'and', 'livelihoods', 'of', 'people', 'and', 'ultimately', 'the', 'very', 'existence', 'of', 'their', 'nations', 'climatechange']
['all', 'populations', 'are', 'at', 'risk', 'of', 'climatechange', 'but', 'small', 'island', 'developing', 'states', 'sids', 'are', 'experiencing', 'a', 'range', 'of', 'acute', 'to', 'longterm', 'risks', 'from', 'extreme', 'heatwaves', 'floods', 'storms', 'amp', 'drought', 'to', 'increased', 'risks', 'of', 'diseases', 'including', 'the'

In [20]:
#inspect the first couple of tweets to check
tweets_json['0']['full_text']

['intern',
 'day',
 'disast',
 'risk',
 'reduct',
 'openwho',
 'launch',
 'multiti',
 'core',
 'curriculum',
 'help',
 'equip',
 'compet',
 'need',
 'work',
 'within',
 'public',
 'health',
 'emerg',
 'respons',
 'start',
 'learn',
 'today',
 'amp',
 'ready4respons']